# Corretoras e Imobiliarias

Vamos pegar as tabelas de todos os corretores e imobiliarias do rio de janeiro

In [25]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from io import StringIO

In [27]:
lista_bairros_zona_norte = [
    "Acari", "Anchieta", "Barros Filho", "Bento Ribeiro", "Brás de Pina", "Campinho", "Cavalcanti", "Cascadura", "Coelho Neto", "Colégio",
    "Complexo do Alemão", "Cordovil", "Costa Barros", "Engenheiro Leal", "Engenho da Rainha", "Guadalupe", "Honório Gurgel", "Inhaúma", "Irajá",
    "Jardim América", "Madureira", "Marechal Hermes", "Oswaldo Cruz", "Parada de Lucas", "Parque Anchieta", "Parque Colúmbia", "Pavuna", "Penha",
    "Penha Circular", "Quintinho Bocaiuva", "Ricardo de Albuquerque", "Rocha Miranda", "Tómas Coelho", "Turiaçu", "Vaz Lobo", "Vicente de Carvalho",
    "Vigário Geral", "Vila da Penha", "Vila Kosmos", "Vista Alegre" 
]
lista_bairros_ilha_gov_leopoldina = [
    "Bomsucesso", "Bancários", "Cacuia", "Cidade Universitária", "Cocotá", "Freguesia", "Galeão", "Jardim Carioca", "Jardim Guanabara", "Maré",
    "Moneró", "Olaria", "Pitangueiras", "Portuguesa", "Praia da Bandeira", "Ramos", "Ribeira", "Tauá", "Zumbi" 
]
lista_bairros_tijuca = [
    "Alto da Boa Vista", "Andaraí", "Grajaú", "Maracanã", "Praça da Bandeira", "Tijuca", "Vila Isabel", 
]
lista_bairros_meier = [
    "Abolição", "Agua Santa", "Cachambi", "Del Castilho", "Encantado", "Engenho de Dentro", "Engenho Novo", "Higienópolis", "Jacaré", "Jacarezinho",
    "Lins de Vasconcelos", "Manguinhos", "Maria da Graça", "Méier", "Piedade", "Pilares", "Riachuelo", "Rocha", "Sampaio", "São Francisco Xavier",
    "Todos os Santos",  
]

In [28]:
lista_bairros = lista_bairros_zona_norte + lista_bairros_ilha_gov_leopoldina + lista_bairros_tijuca + lista_bairros_meier
dict_bairros = {}

In [29]:
def baixar_tabela_por_bairro(bairro):
    url = 'https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php'
    driver = webdriver.Edge()

    #Abrir pagina
    driver.get(url)
    # Localiza os campos do formulário e preenche-os
    campo_bairro = driver.find_element("id", "input-27")
    campo_bairro.send_keys(bairro)

    # Envie o formulário
    campo_bairro.send_keys(Keys.RETURN)

    # Aguarde um tempo para a página carregar completamente
    time.sleep(2)

    # Encontra a tabela na página
    try:
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        print(f"Não existe nenhuma corretora/imobiliaria em {bairro}!")
        print(f"Verifique se o nome do bairro esta certo! {bairro}\n")
        driver.quit()
        return pd.DataFrame()
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        return df

In [30]:
for i in range(10):
    bairro = lista_bairros[i]
    tabela_bairro = baixar_tabela_por_bairro(bairro)
    dict_bairros.update({bairro:tabela_bairro})

In [8]:
tabela_bairro_tijuca = baixar_tabela_por_bairro("Tijuca")

dict_bairros.update({"Tijuca":tabela_bairro_tijuca})

Não existe nenhuma corretora/imobiliaria em Tijuca!
Verifique se o nome do bairro esta certo! Tijuca



In [31]:
dict_bairros_edit = dict_bairros.copy()

In [ ]:
'''
dict_imobiliaria_ativa_disponivel = {}
acari = dict_bairros_edit['Acari']
display(acari['Situação'].value_counts(normalize=True).map('{:.1%}'.format))

for i in acari:
    print(i)
'''
for bairro, tb in dict_bairros_edit.items():
    print(bairro)

In [ ]:
def bairros_ativos_disponiveis(bairros):
    novo_dict = {}
    for bairro_nome, bairro_tabela in bairros.items():
        print(bairro_nome)
        if not bairro_tabela.empty:
            nova_tabela = bairro_tabela.loc[bairro_tabela["Situação"] == "ATIVO"]
            nova_tabela = nova_tabela.loc[bairro_tabela["Certidão de Regularidade"] == "REGULAR"]
            novo_dict.update({bairro_nome:nova_tabela})
        else:
            print("Vazio",bairro_nome)
    return novo_dict.copy()

print(len(dict_bairros_edit))
dict_imobiliaria_ativa_disponivel = bairros_ativos_disponiveis(dict_bairros_edit)
print(len(dict_imobiliaria_ativa_disponivel))

In [ ]:

def descobrir_cnpj(razao_social):
    url = "https://cnpj.linkana.com"
    driver = webdriver.Edge()
    driver.get(url)
    
    time.sleep(1)

    campo_razao_social = driver.find_element("id", "q")
    campo_razao_social.send_keys(razao_social)

    # Envie o formulário
    campo_razao_social.send_keys(Keys.RETURN)
    time.sleep(5)

    
tabela_acari = dict_imobiliaria_ativa_disponivel['Acari']

display(tabela_acari)

for linha in tabela_acari.itertuples():
    
    razao_social = linha.Identificação
    razao_social = razao_social.replace(" PF P", "")
    
    print(razao_social)

## Conectar Google Sheets 

In [35]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gspread_dataframe import set_with_dataframe
import gspread

In [36]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1K1f9cY6HhO-iSG2Yop97XnMZlbDmYJihXISnzXq4w34"
SAMPLE_RANGE_NAME = "Class Data!A2:E"


def create(title, dataframe):
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)
    
    # Create the spreadsheet
    spreadsheet = {"properties": {"title": title}}
    spreadsheet = (
        service.spreadsheets()
        .create(body=spreadsheet, fields="spreadsheetId")
        .execute()
    )
    spreadsheet_id = spreadsheet.get("spreadsheetId")
    print(f"Spreadsheet ID: {spreadsheet_id}")
    
    # Insert DataFrame data into the spreadsheet
    if isinstance(dataframe, pd.DataFrame):
        gc = gspread.service_account()
        sh = gc.open_by_key(spreadsheet_id)
        worksheet = sh.sheet1  # Assuming data goes to the first sheet
        set_with_dataframe(worksheet, dataframe)  # Set DataFrame into the worksheet
        print("Data inserted successfully.")
    else:
        print("Invalid DataFrame.")
    
    return spreadsheet_id
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  acari = dict_bairros_edit["Acari"]
  create("Acari", acari)

Spreadsheet ID: 1BOfdWtUCIrzLP0RVR2YM4FIWCRI73ZyG-EZoINMrFos


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cegli\\AppData\\Roaming\\gspread\\service_account.json'

In [21]:
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
from gspread_dataframe import set_with_dataframe
import gspread


def create(title, dataframe):
    """
    Creates the Sheet the user has access to and inserts DataFrame data.
    Load pre-authorized user credentials from the environment.
    """
    creds, _ = google.auth.default()
    try:

    except HttpError as error:
        print(f"An error occurred: {error}")
        return error


if __name__ == "__main__":
    # Pass: title and DataFrame
    create("Acari", acari)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.